In [1]:
from dataset_io import *
from classifiers import *
from tqdm import tqdm

In [2]:
#np.save("cross_validated_scenes",cross_validated_scenes())
cross_validated_scenes = np.load("cross_validated_scenes.npy", allow_pickle=True)

In [ ]:
grad_metrics=[]
color_metrics=[]

for fold in tqdm(cross_validated_scenes):
    train_scenes, test_scenes, name = fold
    print(name)
    acc, prec, rec, f1, _ = classify_gradient(train_scenes, test_scenes, name, True)
    grad_metrics.append([acc, prec, rec, f1])
    res = classify_colors(train_scenes, test_scenes, name, True)
    color_metrics.append(res)
    
np.save("color_metrics", color_metrics)
np.save("grad_metrics", grad_metrics)

  0%|          | 0/4 [00:00<?, ?it/s]

fold_0
blue
1
2
3
31


In [ ]:
# color_metrics = np.mean(color_metrics)
# grad_metrics = np.mean(grad_metrics)
print(color_metrics)
print(grad_metrics)

In [ ]:
-1. 4 features per all scenes available
-2. get folds
3. for each fold set it as test and load features for others (concat)
4. train models on loaded features
5. pass models to algorithm
6. run algorithm to predict single test fold
7. save predictions in list
8. mean of predctions